In [4]:
import numpy as np
from pathlib import Path

from grl.utils import load_info
from definitions import ROOT_DIR

In [23]:
result_path = Path(ROOT_DIR, 'results', 'tmaze_5_two_thirds_up_mi_pi(pi)_miit(1)_s(2029)_Tue Oct 25 13:44:42 2022.npy')
info = load_info(result_path)

In [24]:
print(f"initial policy, after first improvement: \n {info['logs']['initial_improvement_policy']}")
val_key = 'v'
for k, v in info['logs'].items():
    
    if 'discrep' in k:
        print(f"{k}, for {val_key}: {v[val_key]}")

initial policy, after first improvement: 
 [[0.025 0.025 0.925 0.025]
 [0.025 0.025 0.925 0.025]
 [0.025 0.025 0.925 0.025]
 [0.925 0.025 0.025 0.025]
 [0.925 0.025 0.025 0.025]]
initial_discrep, for v: [2.11949773e-12 2.11811213e-12 2.29922601e-11 9.11928222e-05
 0.00000000e+00]
initial_improvement_discrep, for v: [9.27154724e-01 9.27154723e-01 7.00337419e-02 2.16893259e-04
 0.00000000e+00]
initial_mem_discrep, for v: [5.09499888e-01 2.15675385e-01 5.09499886e-01 5.30591803e-01
 8.44972611e-02 7.43633085e-03 2.03360892e-04 7.56783756e-04
 0.00000000e+00 0.00000000e+00]
final_mem_discrep, for v: [7.93084101e-01 7.95718966e-01 7.93084103e-01 7.93404292e-01
 1.78637897e-01 2.39134126e-01 2.47931606e-04 2.46889445e-04
 0.00000000e+00 0.00000000e+00]


In [25]:
info['logs']['final_outputs']

{'v': DeviceArray([0.92838797, 0.92986551, 1.00524848, 1.00506865, 1.12827694,
              1.03828907, 1.91906449, 1.91988746, 0.        , 0.        ],            dtype=float64),
 'q': DeviceArray([[0.8367076 , 0.83680508, 0.90472302, 0.9045906 , 0.93730241,
               0.93460502, 1.94998342, 1.95092514, 0.        , 0.        ],
              [0.83626351, 0.83683168, 0.90457312, 0.90469461, 0.94092138,
               0.93471616, 1.95001658, 1.94907481, 0.        , 0.        ],
              [0.93583371, 0.93740884, 1.0134071 , 1.01320968, 1.14371202,
               1.04832221, 1.72719211, 1.7272622 , 0.        , 0.        ],
              [0.8367002 , 0.83685635, 0.90458019, 0.90470268, 0.93550905,
               0.87431961, 0.93479078, 0.93493094, 0.        , 0.        ]],            dtype=float64)}

In [28]:
info['agent'].memory

DeviceArray([[[[0.12885553, 0.87114447],
               [0.055553  , 0.944447  ]],

              [[0.9962365 , 0.0037635 ],
               [0.17808089, 0.82191911]],

              [[0.03509428, 0.96490572],
               [0.00178858, 0.99821142]],

              [[0.23984757, 0.76015243],
               [0.82223249, 0.17776751]],

              [[0.77973611, 0.22026389],
               [0.02600625, 0.97399375]]],


             [[[0.4628177 , 0.5371823 ],
               [0.03554768, 0.96445232]],

              [[0.07003799, 0.92996201],
               [0.82069188, 0.17930812]],

              [[0.07977886, 0.92022114],
               [0.00316087, 0.99683913]],

              [[0.97342636, 0.02657364],
               [0.65634647, 0.34365353]],

              [[0.88163664, 0.11836336],
               [0.13546647, 0.86453353]]],


             [[[0.0169597 , 0.9830403 ],
               [0.0364084 , 0.9635916 ]],

              [[0.97478494, 0.02521506],
               [0.97234733, 0.0